by Yigao Li

In [13]:
import json
import numpy as np
import numpy.random as rd
from ipywidgets import widgets
from IPython.display import HTML, Javascript, display

def json_numpy_serializer(o):
    if isinstance(o, np.ndarray):
        return o.tolist()
    raise TypeError("{} of type {} is not JSON serializable".format(repr(o), type(o)))

def jsglobal(**params):
    code = [];
    for name, value in params.items():
        jsdata = json.dumps(value, default=json_numpy_serializer)
        code.append("window.{}={};".format(name, jsdata))
    display(Javascript("\n".join(code)))

In [14]:
%%javascript
require.config({
    paths: {
        mathBox:'//cdn.rawgit.com/unconed/mathbox/eaeb8e15/build/mathbox-bundle'
    }
});

window.with_mathbox = function(element, func) {
    require(['mathBox'], function(){
        var mathbox = mathBox({
          plugins: ['core', 'controls', 'cursor', 'mathbox'],
          controls: { klass: THREE.OrbitControls },
          mathbox: {inspect: false},
          element: element[0],
          loop: {start: false},
            
        });
        
        var three = mathbox.three;
        three.renderer.setClearColor(new THREE.Color(0xFFFFFF), 1.0);
        three.camera.position.set(-1, 1, 2);
        three.controls.noKeys = true;
        
        three.element.style.height = "400px";
        three.element.style.width = "100%";
        
        function isInViewport(element) {
          var rect = element.getBoundingClientRect();
          var html = document.documentElement;
          var w = window.innerWidth || html.clientWidth;
          var h = window.innerHeight || html.clientHeight;
          return rect.top < h && rect.left < w && rect.bottom > 0 && rect.right > 0;
        }
        
        // Running update/render loop only for visible plots.
        var intervalId = setInterval(function(){
            if (three.element.offsetParent === null) {
                clearInterval(intervalId);
                three.destroy();
                return;
            }
            var visible = isInViewport(three.canvas);
            if (three.Loop.running != visible) {
                visible? three.Loop.start() : three.Loop.stop();
            }
        }, 100);

        func(mathbox);
        
        window.dispatchEvent(new Event('resize'));
    })
};

// Function for setting up 3D axes and plotting the graph of a 2D function f
window.plotGraph = function(mathbox, f, xlabel='x', ylabel='y', zlabel='f(x,y)', rng=[[-3, 3], [-5, 5], [-3, 3]]) {
    
    var view = mathbox.cartesian({range: rng,
                                  scale: [1, 1, 1]},
                                  {rotation:(t)=>[0, t*0.02, 0]}
                                ).grid({axes: [1, 3]})
     
    view.area({id: 'yaxis',
               width: 1,
               height: 1,
               axes: [1, 3], 
               expr: function (emit, x, y, i, j) {
                   emit(4, 0, 0);
                   emit(0, 0, 0);
               },
               items: 2,
               channels: 3,
              })
        .text({font: 'Helvetica',
               style: 'bold',
               width:  16,
               height: 5,
               depth:  2,
               expr: function (emit, i, j, k, time) {
                   emit(ylabel);
               },
              })
        .label({color: '#000000',
                snap: false,
                outline: 2,
                size: 24,
                offset: [0, -32],
                depth: .5,
                zIndex: 1
               });
              
    view.vector({points: '#yaxis',
                 color: 0x000000,
                 width: 9,
                 start: true
                });
    
    view.area({id: 'xaxis',
               width: 1,
               height: 1,
               axes: [1, 3],
               expr: function (emit, x, y, i, j) {
                   emit(0, 0, 4);
                   emit(0, 0, 0);
               },
               items: 2,
               channels: 3,
              })
        .text({font: 'Helvetica',
               style: 'bold',
               width:  16,
               height: 5,
               depth:  2,
               expr: function (emit, i, j, k, time) {
                   emit(xlabel);
               },
              })
        .label({color: '#000000',
                snap: false,
                outline: 2,
                size: 24,
                offset: [0, -32],
                depth: .5,
                zIndex: 1,
               });
              
    view.vector({points: '#xaxis',
                 color: 0x000000,
                 width: 9,
                 start: true,
                });
    
    view.area({id: 'zaxis',
               width: 1,
               height: 1,
               axes: [1, 3],
               expr: function (emit, x, y, i, j) {
                   emit(0, 4, 0);
                   emit(0, 0, 0);
               },
               items: 2,
               channels: 3,
              })
        .text({font: 'Helvetica',
               style: 'bold',
               width:  16,
               height: 5,
               depth:  2,
               expr: function (emit, i, j, k, time) {
                   emit(zlabel);
               },
              })
        .label({color: '#000000',
                snap: false,
                outline: 2,
                size: 24,
                offset: [0, -32],
                depth: .5,
                zIndex: 1,
               });

    view.vector({points: '#zaxis',
                 color: 0x000000,
                 width: 9,
                 start: true,
                });
    
    var graph = view.area({id:'graph',
                             width: 64,
                             height: 64,
                             axes: [1, 3],
                             expr: function (emit, y, x, i, j) {
                                 emit(y, f(x, y), x);
                             },
                             items: 1,
                             channels: 3,
                            });
    
    view.surface({shaded: true,
                  lineX: true,
                  lineY: true,
                  points: graph,
                  color: 0x0000FF,
                  width: 1,
                 });
    
    return view;
};

window.addSegment = function(view, p0, p1, col) {
    
    view.array({width: 128,
                expr: function (emit, i, time) {
                    var b = i/128;
                    var a = 1-b;
                    emit(a*p0[1] + b*p1[1], a*p0[2] + b*p1[2], a*p0[0] + b*p1[0]);
                },
                channels: 3,
               });

    view.line({color: col,
               width: 10,
               size: 2.5,
               stroke: 'dotted',
               start: false,
               end: false,
              });
    
};

window.addPoint = function(view, p, col, label) {
    view.array({width: 4,
                items: 2,
                channels: 3,
                expr: function (emit, i, t) {
                    emit(p[1],p[2],p[0]);
                },
               })
        .point({color:  col,
                points: '<',
                size: 15,
                depth: .5,
                zBias: 50,
               })
        .text({font: 'Helvetica',
               style: 'bold',
               width:  16,
               height: 5,
               depth:  2,
               expr: function (emit, i, j, k, time) {
                   emit(label);
               },
              })
        .label({color: col,
                snap: false,
                outline: 2,
                size: 24,
                offset: [0, -32],
                depth: .5,
                zIndex: 1,
               });
};

window.addCurve = function(view, ab, x, y, z, col) {
    
    view.array({width: 128,
                expr: function (emit, i, time) {
                    var t = (ab[1]-ab[0])*(i/128) + ab[0];
                    emit(y(t), z(t), x(t));
                },
                channels: 3,
               });

    view.line({color: col,
               width: 20,
               size: 2.5,
               start: true,
               end: true,
              });
};

window.addClosedCurve = function(view, ab, x, y, z, col) {
    
    view.array({width: 128,
                expr: function (emit, i, time) {
                    var t = (ab[1]-ab[0])*(i/128) + ab[0];
                    emit(y(t), z(t), x(t));
                },
                channels: 3,
               });

    view.line({color: col,
               width: 20,
               size: 2.5,
               start: false,
               end: false,
              });
};


window.addSurface = function(view, ab, cd, x, y, z, col, opa) {
    
    view.matrix({width: 64,
                 height: 64,
                 expr: function (emit, i, j, time) {
                     var p = (ab[1]-ab[0])*(i/64) + ab[0];
                     var q = (cd[1]-cd[0])*(j/64) + cd[0];
                     emit(y(p, q), z(p, q), x(p, q));
                 },
                 items: 1,
                 channels: 3
                })
        .surface({shaded: true,
                  lineX: false,
                  lineY: false,
                  color: col,
                  width: 1,
                  opacity: opa
                 });
}

window.addSequence = function(view, seq, col) {
    
    var idx = 0;
    var d = new Date();
    var start = d.getTime();
    view.array({width: 1,
                expr: function (emit, i, time) {
                    var nd = new Date();
                    var now = nd.getTime();
                    if (1000 < now-start) {
                        idx = idx + 1;
                        if (seq.length <= idx) {
                            idx = 0;
                        }
                        start = now;
                    }
                    emit(seq[idx][1], seq[idx][2], seq[idx][0]);    
                },
                items: 1,
                channels: 3
               }) 
        .point({color:  col,
                points: '<',
                size: 15,
                depth: .5,
                zBias: 50,
                });
    
}

<IPython.core.display.Javascript object>

# Exercise 1
$$f(x,y)=x^2+y^2$$

In [15]:
%%javascript

with_mathbox(element, function(mathbox) {
    
    var fcn = function(x, y) {
        return (Math.pow(x,2) + Math.pow(y,2));
    }
    
    var view = plotGraph(mathbox, fcn);
    
})

<IPython.core.display.Javascript object>

$$f(x,y)=x^2$$

In [5]:
%%javascript

with_mathbox(element, function(mathbox) {
    
    var fcn = function(x, y) {
        return (Math.pow(x,2));
    }
    
    var view = plotGraph(mathbox, fcn);
    
})

<IPython.core.display.Javascript object>

$$f(x,y)=x^2-y^2$$

In [6]:
%%javascript

with_mathbox(element, function(mathbox) {
    
    var fcn = function(x, y) {
        return (Math.pow(x,2) - Math.pow(y,2));
    }
    
    var view = plotGraph(mathbox, fcn);
    
})

<IPython.core.display.Javascript object>

$$f(x,y)=-x^2$$

In [7]:
%%javascript

with_mathbox(element, function(mathbox) {
    
    var fcn = function(x, y) {
        return (- Math.pow(x,2));
    }
    
    var view = plotGraph(mathbox, fcn);
    
})

<IPython.core.display.Javascript object>

$$f(x,y)=-x^2-y^2$$

In [8]:
%%javascript

with_mathbox(element, function(mathbox) {
    
    var fcn = function(x, y) {
        return (- Math.pow(x,2) - Math.pow(y,2));
    }
    
    var view = plotGraph(mathbox, fcn);
    
})

<IPython.core.display.Javascript object>

Functions $f(x,y)=x^2+y^2$ and $f(x,y)=x^2$ are convex. $f(x,y)-x^2+y^2$ is strictly convex.

# Exercise 2
$$f(x,y)=x^2+y^2\\
\frac{\partial f}{\partial x}(x,y)=2x\\
\frac{\partial f}{\partial y}(x,y)=2y\\$$
Let ${\bf x}=(x_1,x_2),{\bf y}=(y_1,y_2)\in\mathbb{R}^2$ and ${\bf x}\neq{\bf y}$  
$$f(x_1,x_2)=x_1^2+x_2^2$$
$$\begin{aligned}
f(y_1,y_2)+\frac{\partial f}{\partial x}(y_1,y_2)(x_1-y_1)+\frac{\partial f}{\partial y}(y_1,y_2)(x_2-y_2)&=y_1^2+y_2^2+2y_1(x_1-y_1)+2y_2(x_2-y_2)\\
&=2x_1y_1+2x_2y_2-y_1^2-y_2^2\\
\end{aligned}$$
$$\begin{aligned}
f(x_1,x_2)-f(y_1,y_2)-\frac{\partial f}{\partial x}(y_1,y_2)(x_1-y_1)-\frac{\partial f}{\partial y}(y_1,y_2)(x_2-y_2)&=x_1^2+x_2^2-2x_1y_1-2x_2y_2+y_1^2+y_2^2\\
&=(x_1-y_1)^2+(x_2-y_2)^2\\
&>0\\
\end{aligned}$$
So $f(x,y)=x^2+y^2$ is strictly convex.

$$f(x,y)=x^2\\
\frac{\partial f}{\partial x}(x,y)=2x\\
\frac{\partial f}{\partial y}(x,y)=0\\$$
Let ${\bf x}=(x_1,x_2),{\bf y}=(y_1,y_2)\in\mathbb{R}^2$ and ${\bf x}\neq{\bf y}$  
$$f(x_1,x_2)=x_1^2$$
$$\begin{aligned}
f(y_1,y_2)+\frac{\partial f}{\partial x}(y_1,y_2)(x_1-y_1)+\frac{\partial f}{\partial y}(y_1,y_2)(x_2-y_2)&=y_1^2+2y_1(x_1-y_1)\\
&=2x_1y_1-y_1^2\\
\end{aligned}$$
$$\begin{aligned}
f(x_1,x_2)-f(y_1,y_2)-\frac{\partial f}{\partial x}(y_1,y_2)(x_1-y_1)-\frac{\partial f}{\partial y}(y_1,y_2)(x_2-y_2)&=x_1^2-2x_1y_1+y_1^2\\
&=(x_1-y_1)^2\\
&\geq0\\
\end{aligned}$$
So $f(x,y)=x^2$ is convex.

$$f(x,y)=x^2-y^2\\
\frac{\partial f}{\partial x}(x,y)=2x\\
\frac{\partial f}{\partial y}(x,y)=-2y\\$$
Let ${\bf x}=(x_1,x_2),{\bf y}=(y_1,y_2)\in\mathbb{R}^2$ and ${\bf x}\neq{\bf y}$  
$$f(x_1,x_2)=x_1^2-x_2^2$$
$$\begin{aligned}
f(y_1,y_2)+\frac{\partial f}{\partial x}(y_1,y_2)(x_1-y_1)+\frac{\partial f}{\partial y}(y_1,y_2)(x_2-y_2)&=y_1^2-y_2^2+2y_1(x_1-y_1)-2y_2(x_2-y_2)\\
&=2x_1y_1-2x_2y_2-y_1^2+y_2^2\\
\end{aligned}$$
Take ${\bf x}=(x_1,x_2)=(9,9)$ and ${\bf y}=(y_1,y_2)=(2,1)$ 
$$\begin{aligned}
f(x_1,x_2)-f(y_1,y_2)-\frac{\partial f}{\partial x}(y_1,y_2)(x_1-y_1)-\frac{\partial f}{\partial y}(y_1,y_2)(x_2-y_2)&=x_1^2-2x_1y_1+y_1^2-x_2^2+2x_2y_2-y_2^2\\
&=(x_1-y_1)^2-(x_2-y_2)^2\\
&=7^2-8^2\\
&=-15\\
&<0\\
\end{aligned}$$
So $f(x,y)=x^2-y^2$ is not convex.

$$f(x,y)=-x^2\\
\frac{\partial f}{\partial x}(x,y)=-2x\\
\frac{\partial f}{\partial y}(x,y)=0\\$$
Let ${\bf x}=(x_1,x_2),{\bf y}=(y_1,y_2)\in\mathbb{R}^2$ and ${\bf x}\neq{\bf y}$  
$$f(x_1,x_2)=-x_1^2$$
$$\begin{aligned}
f(y_1,y_2)+\frac{\partial f}{\partial x}(y_1,y_2)(x_1-y_1)+\frac{\partial f}{\partial y}(y_1,y_2)(x_2-y_2)&=-y_1^2+2y_1(x_1-y_1)\\
&=y_1^2-2x_1y_1\\
\end{aligned}$$
$$\begin{aligned}
f(x_1,x_2)-f(y_1,y_2)-\frac{\partial f}{\partial x}(y_1,y_2)(x_1-y_1)-\frac{\partial f}{\partial y}(y_1,y_2)(x_2-y_2)&=-x_1^2+2x_1y_1-y_1^2\\
&=-(x_1-y_1)^2\\
&\leq0\\
\end{aligned}$$
So $f(x,y)=-x^2$ is not convex.

$$f(x,y)=-x^2-y^2\\
\frac{\partial f}{\partial x}(x,y)=-2x\\
\frac{\partial f}{\partial y}(x,y)=-2y\\$$
Let ${\bf x}=(x_1,x_2),{\bf y}=(y_1,y_2)\in\mathbb{R}^2$ and ${\bf x}\neq{\bf y}$  
$$f(x_1,x_2)=-x_1^2-x_2^2$$
$$\begin{aligned}
f(y_1,y_2)+\frac{\partial f}{\partial x}(y_1,y_2)(x_1-y_1)+\frac{\partial f}{\partial y}(y_1,y_2)(x_2-y_2)&=-y_1^2-y_2^2-2y_1(x_1-y_1)-2y_2(x_2-y_2)\\
&=-2x_1y_1-2x_2y_2+y_1^2+y_2^2\\
\end{aligned}$$
$$\begin{aligned}
f(x_1,x_2)-f(y_1,y_2)-\frac{\partial f}{\partial x}(y_1,y_2)(x_1-y_1)-\frac{\partial f}{\partial y}(y_1,y_2)(x_2-y_2)&=-x_1^2+-x_2^2+2x_1y_1+2x_2y_2-y_1^2-y_2^2\\
&=-(x_1-y_1)^2-(x_2-y_2)^2\\
&<0\\
\end{aligned}$$
So $f(x,y)=x^2+y^2$ is not convex.

# Exercise 3
(a) $\forall\epsilon>0$, take $\delta=\epsilon$.
$$\begin{aligned}
|\sqrt{(x-0)^2+(y-0)^2}|&<\delta=\epsilon\\
|\frac{y^2}{\sqrt{x^2+y^2}}|<|\frac{x^2+y^2}{\sqrt{x^2+y^2}}|&<\\
\end{aligned}$$
$$|\frac{y^2}{\sqrt{x^2+y^2}}-0|<\epsilon$$
So $f(x,y)=\frac{y^2}{\sqrt{x^2+y^2}}$ is continuous at $(0,0)$ and $f(0,0)=0$.

(b) Let $t,s\in\mathbb{R}$, $g(t)=\frac{b^2}{\sqrt{a^2+b^2}}t$
$$\begin{aligned}
g(s)+g'(s)(t-s)&=\frac{b^2}{\sqrt{a^2+b^2}}s+\frac{b^2}{\sqrt{a^2+b^2}}(t-s)\\
&=\frac{b^2}{\sqrt{a^2+b^2}}t\\
\end{aligned}$$
$$g(t)=g(s)+g'(s)(t-s)$$
So $g(t)$ is convex.

(c) Take ${\bf x}=(3,-3)$, ${\bf y}=(-3,0)$, $t=\frac{5}{6}$  
$$f((1-t){\bf x} + t{\bf y})=\frac{1}{\sqrt{5}}\\
(1-t)f({\bf x}) + t f({\bf y})=\frac{\sqrt{2}}{4}\\
\frac{1}{\sqrt{5}}>\frac{\sqrt{2}}{4}$$
So $f(x,y)$ is not convex

# Exercise 4
$$\begin{aligned}
f(x_1,x_2)&=-log((1+x_1^2)(1+x_2^2)-x_1^2x_2^2)\\
&=-log(1+x_1^2+x_2^2)\\
\end{aligned}$$
$$\frac{\partial f}{\partial x_1}=-\frac{2x_1}{1+x_1^2+x_2^2}$$
$$\frac{\partial f}{\partial x_2}=-\frac{2x_2}{1+x_1^2+x_2^2}$$
$$\frac{\partial^2f}{\partial x_1^2}=-\frac{2(1-x_1^2+x_2^2)}{(1+x_1^2+x_2^2)^2}$$
$$\frac{\partial^2f}{\partial x_2^2}=-\frac{2(1+x_1^2-x_2^2)}{(1+x_1^2+x_2^2)^2}$$
$$\frac{\partial^2f}{\partial x_1\partial x_2}=\frac{4x_1x_2}{(1+x_1^2+x_2^2)^2}$$
Second order Taylor approximation $p(x_1,x_2)$ at $(1,1)$:
$$\begin{aligned}
p(x_1,x_2)&=f(1,1)+\frac{\partial f}{\partial x_1}(1,1)(x_1-1)+\frac{\partial f}{\partial x_2}(1,1)(x_2-1)+\frac{1}{2}\frac{\partial^2f}{\partial x_1^2}(1,1)(x_1-1)^2+\frac{1}{2}\frac{\partial^2f}{\partial x_2^2}(1,1)(x_2-1)^2+\frac{\partial^2f}{\partial x_1\partial x_2}(1,1)(x_1-1)(x_2-1)\\
&=-log(3)-\frac{2}{3}(x_1-1)-\frac{2}{3}(x_2-1)-\frac{1}{9}(x_1-1)^2-\frac{1}{9}(x_2-1)^2+\frac{4}{9}(x_1-1)(x_2-1)\\
\end{aligned}$$

# Exercise 5
(a) Let $A=\begin{pmatrix}a&b\\b&c\end{pmatrix}$,$B=\begin{pmatrix}d&e\\e&f\end{pmatrix}\in SPD(2)$. Thus, $\forall (x_1,x_2)^T\in\mathbb{R}^2$
$$ax_1^2+2bx_1x_2+cx_2^2\geq0$$
$$dx_1^2+2ex_1x_2+fx_2^2\geq0$$
$$(a+d)x_1^2+2(b+e)x_1x_2+(c+f)x_2^2\geq0$$
So $A+B=\begin{pmatrix}a+d&b+e\\b+e&c+f\end{pmatrix}\in SPD(2)$

(b) Let $A=\begin{pmatrix}a&b\\b&c\end{pmatrix}$,$B=\begin{pmatrix}d&e\\e&f\end{pmatrix}\in SPD(2)$ and $t\in[0,1]$. Then $tA+(1-t)B=\begin{pmatrix}ta+(1-t)d&tb+(1-t)e\\tb+(1-t)e&tc+(1-t)f\end{pmatrix}$
$$(ta+(1-t)d)x_1^2+2(tb+(1-t)e)x_1x_2+(tc+(1-t)f)x_2^2=t(ax_1^2+2bx_1x_2+cx_2^2)+(1-t)(dx_1^2+2ex_1x_2+fx_2^2)\geq0$$
So $SPD(2)$ is a convex subset.

(c) Let $X=\begin{pmatrix}a&b\\c&d\end{pmatrix}$, then $X^T=\begin{pmatrix}a&c\\b&d\end{pmatrix}$.
$$X^TX=\begin{pmatrix}a^2+c^2&ab+cd\\ab+cd&b^2+d^2\end{pmatrix}\in M_2(\mathbb{R})$$
$\forall {\bf x}=(x_1,x_2)^T\in\mathbb{R}^2$
$$\begin{aligned}
{\bf x}^TX^TX{\bf x}&=(a^2+c^2)x_1^2+2(ab+cd)x_1x_2+(b^2+d^2)x_2^2\\
&=(ax_1+bx_2)^2+(cx_1+dx_2)^2\\
&\geq0\\
\end{aligned}$$
So $X^TX\in SPD(2)$

(d) Let $A=\begin{pmatrix}a&b\\b&c\end{pmatrix}$ be positive semidefinite, $B=\begin{pmatrix}d&e\\e&f\end{pmatrix}\in SPD(2)$ be positive definite. Thus, $\forall (x_1,x_2)^T\in\mathbb{R}^2$
$$ax_1^2+2bx_1x_2+cx_2^2\geq0$$
$$dx_1^2+2ex_1x_2+fx_2^2>0$$
$$(a+d)x_1^2+2(b+e)x_1x_2+(c+f)x_2^2>0$$
So $A+B=\begin{pmatrix}a+d&b+e\\b+e&c+f\end{pmatrix}\in SPD(2)$ is positive definite.

(e) Let $A=\begin{pmatrix}a&b\\b&c\end{pmatrix}$ be positive definite, then $\forall {\bf x}=(x_1,x_2)^T\in\mathbb{R}^2$ and ${\bf x}\neq{\bf0}$, $ax_1^2+2bx_1x_2+cx_2^2>0$.  
Assume there exists $A$ that is not invertible, $det(A)=ac-b^2=0\implies ac=b^2\implies b=\sqrt{ac}$  
When $a,b,c=0$, ${\bf x}^TA{\bf x}=0$. Contradiction.  
When $a,b=0$, $c\neq0$, $\exists x_2=0:{\bf x}^TA{\bf x}=0$. Contradiction.  
Similarly, when $b,c=0$, $a\neq0$. Contradiction.  
When $a,b,c\neq0$, $a,c>0$, $ax_1^2+2bx_1x_2+cx_2^2=ax_1^2+2\sqrt{ac}x_1x_2+cx_2^2=(\sqrt{a}x_1+\sqrt{c}x_2)^2$. Let $\frac{x_1}{x_2}=-\sqrt{\frac{c}{a}}$. $ax_1^2+2bx_1x_2+cx_2^2=0$. Contradiction.  
Similarly, when $a,b,c\neq0$, $a,c<0$. Contradiction.  
Overall, $A\in M_2(\mathbb{R})$ is positive definite $\implies$ $A$ is invertible.